In [29]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

In [30]:
SYSTEM_PROMPT_5="""
I want you to act as a software tester.
Given a test scenario and the corresponding use case specification.
Generating test case to test the given test scenario follow the given use case specification description.
When generating test case:
- Optional test steps can be skipped if they are not required to be tested from the test scenario description.
- If the step interacts with the value selection element, if the element has a default value, skip that step, otherwise, choose any value for that step.
The generated test case must follow the following template: 

Title/Description: {A concise description of the purpose of the test case based on the given test scenario.}
Test Objective: {The specific goal or objective of the test.}
Preconditions: {Any necessary conditions that must be met before the test is executed which is mentioned in the use case specification.}
Test Steps: {A step-by-step sequence of actions to perform during the test follow the step declared in the use case }
Expected Results: {The anticipated outcomes or behaviors after executing the given test scenario}

"""

# Also, generate a test case includes test steps based on those information to cover the test scenario.
# Rules for generating test case:
# - If that test case needs login as a specific role to proceed, do not put login at step 1. Instead, put the login as that specific role in the pre condition.
# - If in the use case specification do not describe how to go to the page to do the test case, do not put it in the step. Instead, tell the tester that they need to be in that page in the pre condition.
# - If there are navigation steps in the use case specification, keep it. Do not put it in the precondition or combine it with other steps.
# - If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
# - Generate only test cases that the test scenario's name mentioned. Do not try to cover everything in the use case specification if the test scenario's name does not describe it. 
# - Write the test step clearly, do not write something like "Follow steps from the main flow".
# - If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.

In [31]:
testscenario = """Test scenario: 
Create board with Visibility value 'Public'
"""

In [32]:
usecase="""    Use case specification: 
use case name: create a workspace
precondition: user has been logged in.
main flow: 
Step 1: User click button 'Create' on the navbar.
Step 2: system show up drop down with options: Create board, start with a template, create workspace.
Step 3: user click create workspace.
Step 4: System open a diaglog 'Build a workspace' with text field 'Workspace name', combo box 'Workspace type' with options: Operations, Human Resources, Engineering, Marketing, Small Bussiness, Sales, Education, Other and text field 'Workspace description'. 'Workspace name' and 'Workspace type' are complusary and 'Workspace description' is optional. The button 'Continue' is disable.
Step 5: User input 'Workspace name' is the name of user's company, team or orgainization. 
Step 5: User select values for 'Workspace type'.
Step 6: The button 'Continue' is clickable.
Step 7: User input 'Workspace description'.
Step 8: User click button 'Continue'.
Step 9: System open 'Invite' dialog with text field 'Workspace members', button 'copy link', button 'Invite' with is disable and button 'Do it later'. a valid 'Workspace members' is a name of application's user or is an email. The 'Workspace members' can only invite up to 9 member.
Step 10: User click button 'Do it later' to create Workspace without inviting anyone.
Step 11: New workspace is created and system redirect user to that new workspace page.

Alternative flow:
Alternative flow 1: User skip 'Workspace description',
At step 7 of the basic flow: User skip the 'Workspace description' field.

Alternative flow 2: User invite member by their username.
At step 10 of the basic flow: User enter a valid username of application.
Step 11: button 'Invite' is clickable.
Step 12: User click button 'Invite'.
Step 13: New workspace is created and system redirect user to that new workspace page.

Alternative flow 3: User invite member by link.
At step 10 of the basic flow: User click button 'copy link' .
Step 11: The link is copied into clipboard.
Step 12: User sent link to one another person on other social media.
Step 13: User click button 'Invite'.

Alternative flow 4: User invite member by their email.
At step 10 of the basic flow: User enter a valid email.
Step 11: button 'Invite' is clickable.
Step 12: User click button 'Invite'.
Step 13:New workspace is created and system redirect user to that new workspace page.


Exception flow:
Exception flow 1: User invite more than 9 member to "Workspace members".
At step 10 of the basic flow: User enter 9 valid username/email/member of application to "Workspace members".
Step 11: System sent notification that can not invite more people.


Postcondition: New workspace is created and system redirect user to that new workspace page.
"""

In [33]:
promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT_5},
    { "role": "user", "content": testscenario+usecase}
  ]
gpt_response = ask(promptTestCase, client, model)
print(gpt_response)

Title/Description: Create a Public Workspace without Inviting Members

Test Objective: To verify that a user can successfully create a public workspace without inviting any members.

Preconditions: User has been logged in.

Test Steps:
1. Click the 'Create' button on the navbar.
2. From the dropdown, click 'Create workspace'.
3. In the 'Build a workspace' dialog, enter a valid 'Workspace name'.
4. From the 'Workspace type' combo box, select 'Public'.
5. Verify that the 'Continue' button becomes clickable.
6. (Optional) Enter a description in the 'Workspace description' field.
7. Click the 'Continue' button.
8. In the 'Invite' dialog, click the 'Do it later' button.

Expected Results: 
- The system should create the new workspace and redirect the user to the new workspace page without inviting any members.
- The workspace should be set with the Visibility value 'Public'.


In [34]:
promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT_5},
    { "role": "user", "content": testscenario+usecase}
  ]
gpt_response = ask(promptTestCase, client, model)
print(gpt_response)

Title/Description: Create a Public Workspace without Inviting Members

Test Objective: To verify that a user can successfully create a public workspace without inviting any members.

Preconditions: User has been logged in.

Test Steps:
1. Click the 'Create' button on the navbar.
2. From the dropdown, click 'Create workspace'.
3. In the 'Build a workspace' dialog, enter a valid 'Workspace name'.
4. From the 'Workspace type' combo box, select 'Public' (assuming 'Public' is an option, if not, select any available option as the scenario specifies creating a board with Visibility value 'Public', but the use case does not include this step directly).
5. Verify the 'Continue' button becomes clickable.
6. Click the 'Continue' button.
7. In the 'Invite' dialog, click the 'Do it later' button.

Expected Results: The system should create the new workspace and redirect the user to the new workspace page without inviting any members.


In [35]:
promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT_5},
    { "role": "user", "content": testscenario+usecase}
  ]
gpt_response = ask(promptTestCase, client, model)
print(gpt_response)

Title/Description: Create a Public Workspace without Inviting Members

Test Objective: To verify that a user can successfully create a public workspace without inviting any members.

Preconditions: User has been logged in.

Test Steps:
1. Click the 'Create' button on the navbar.
2. From the dropdown, click 'Create workspace'.
3. In the 'Build a workspace' dialog, enter a valid 'Workspace name'.
4. From the 'Workspace type' combo box, select 'Public' (assuming 'Public' is an option, if not, select any available type as the scenario specifies creating a board with Visibility value 'Public', but the use case does not include this step directly).
5. Verify the 'Continue' button becomes clickable.
6. (Optional) Enter a description in the 'Workspace description' text field.
7. Click the 'Continue' button.
8. In the 'Invite' dialog, click the 'Do it later' button.

Expected Results: A new public workspace is created without inviting any members, and the system redirects the user to the new wo